<a href="https://colab.research.google.com/github/pjazayeri/stylegan-encoder/blob/master/style_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating High Rez GAN Faces with Google CoLab

This notebook demonstrates how to run [NVidia StyleGAN](https://github.com/NVlabs/stylegan) inside of a Google CoLab notebook.  I suggest you use this to generate GAN faces from a pretrained model.  If you try to train your own, you will run into compute limitations of Google CoLab.

Make sure to run this code on a GPU instance.  GPU is assumed.

# Instructions

First, map your G-Drive, this is where your GANs will be written to.

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) #Get access to google drive using personal account (pj1)

In [0]:
rm  -r stylegan-encoder #Remove the repo if it exists

In [0]:
!git clone https://github.com/pjazayeri/stylegan-encoder.git #clone the repo

In [0]:
import sys
sys.path.insert(0, "/content/stylegan-encoder")

import dnnlib #verify that the repo was cloned

In [21]:
!ls /content/drive/My\ Drive/raw_images/ #put raw images in the /raw_images folder

aryana2.png  aryana.png


In [18]:
!python /content/stylegan-encoder/align_images.py /content/drive/My\ Drive/raw_images/ /content/drive/My\ Drive/aligned_images/ #use align_images.py to crop and align raw images

Using TensorFlow backend.


In [0]:
!ls /content/drive/My\ Drive/aligned_images #check which imaged have been aligned to just detected faces.

In [0]:
!python /content/stylegan-encoder/encode_images.py /content/drive/My\ Drive/aligned_images/ /content/drive/My\ Drive/generated_images/ /content/drive/My\ Drive/latent_representations/

Add the Stylegan folder to Python so that you can import it.

The code below is baed on code from NVidia.  This actually generates your images.

In [0]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for reproducing the figures of the StyleGAN paper using pre-trained generators."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

#----------------------------------------------------------------------------
# Helpers for loading and using pre-trained generators.

url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
url_celebahq    = 'https://drive.google.com/uc?id=1MGqJl28pN4t7SAtSrPdSRJSQJqahkzUf' # karras2019stylegan-celebahq-1024x1024.pkl
url_bedrooms    = 'https://drive.google.com/uc?id=1MOSKeGF0FJcivpBI7s63V9YHloUTORiF' # karras2019stylegan-bedrooms-256x256.pkl
url_cars        = 'https://drive.google.com/uc?id=1MJ6iCfNtMIRicihwRorsM3b7mmtmK9c3' # karras2019stylegan-cars-512x384.pkl
url_cats        = 'https://drive.google.com/uc?id=1MQywl0FNt6lHu8E_EUqnRbviagS7fbiJ' # karras2019stylegan-cats-256x256.pkl

synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

_Gs_cache = dict()

def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]

#----------------------------------------------------------------------------
# Figures 2, 3, 10, 11, 12: Multi-resolution grid of uncurated result images.

def draw_uncurated_result_figure(png, Gs, cx, cy, cw, ch, rows, lods, seed):
    print(png)
    latents = np.random.RandomState(seed).randn(sum(rows * 2**lod for lod in lods), Gs.input_shape[1])
    images = Gs.run(latents, None, **synthesis_kwargs) # [seed, y, x, rgb]

    canvas = PIL.Image.new('RGB', (sum(cw // 2**lod for lod in lods), ch * rows), 'white')
    image_iter = iter(list(images))
    for col, lod in enumerate(lods):
        for row in range(rows * 2**lod):
            image = PIL.Image.fromarray(next(image_iter), 'RGB')
            image = image.crop((cx, cy, cx + cw, cy + ch))
            image = image.resize((cw // 2**lod, ch // 2**lod), PIL.Image.ANTIALIAS)
            canvas.paste(image, (sum(cw // 2**lod for lod in lods[:col]), row * ch // 2**lod))
    canvas.save(png)

#----------------------------------------------------------------------------
# Figure 3: Style mixing.

def draw_style_mixing_figure(png, Gs, w, h, src_seeds, dst_seeds, style_ranges,latents):
    print(png)
    src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
    dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)
    
    #print([range(0,4)]*3+[range(4,8)]*2+[range(8,18)])
    #print(src_latents)
    
    dlatents = np.stack(latents)
    
    print(src_latents.shape)
    
    src_dlatents = dlatents #Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
    dst_dlatents = dlatents #Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
    
    print(src_dlatents.shape)
    
    src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
    dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

    canvas = PIL.Image.new('RGB', (w * (len(dlatents) + 1), h * (len(dlatents) + 1)), 'white')
    for col, src_image in enumerate(list(src_images)):
        canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
    for row, dst_image in enumerate(list(dst_images)):
        canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
        row_dlatents = np.stack([dst_dlatents[row]] * len(dlatents))
        row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
    canvas.save(png)

#----------------------------------------------------------------------------
# Main program.

def main():
    tflib.init_tf()
    os.makedirs(config.result_dir, exist_ok=True)
    neema = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/neema_01.npy'))
    neema01 = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/1_01.npy'))
    alex = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/2_01.npy'))
    herosh = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/3_01.npy'))
    eman = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/4_01.npy'))
    nk = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/nk_01.npy'))
    hilda = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/hilda_01.npy'))
    sarah = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/sarah_01.npy'))
    neda = np.load(os.path.join(config.result_dir, f'/content/drive/My Drive/latent_representations/neda_01.npy'))

    arg_latents = [neema,neda,hilda,sarah]

    #draw_uncurated_result_figure(os.path.join(config.result_dir, 'figure02-uncurated-ffhq.png'), load_Gs(url_ffhq), cx=0, cy=0, cw=1024, ch=1024, rows=3, lods=[0,1,2,2,3,3], seed=5)
    draw_style_mixing_figure(os.path.join(config.result_dir, f'/content/drive/My Drive/images/figure03-style-mixing.png'), load_Gs(url_ffhq), w=1024, h=1024, src_seeds=[123,701,687,615], dst_seeds=[888,829,1898,1733,1614], style_ranges=[range(0,4)]*3+[range(4,8)]*2+[range(8,18)], latents=arg_latents)
    #draw_noise_detail_figure(os.path.join(config.result_dir, 'figure04-noise-detail.png'), load_Gs(url_ffhq), w=1024, h=1024, num_samples=100, seeds=[1157,1012])
    #draw_noise_components_figure(os.path.join(config.result_dir, 'figure05-noise-components.png'), load_Gs(url_ffhq), w=1024, h=1024, seeds=[1967,1555], noise_ranges=[range(0, 18), range(0, 0), range(8, 18), range(0, 8)], flips=[1])
    #draw_truncation_trick_figure(os.path.join(config.result_dir, 'figure08-truncation-trick.png'), load_Gs(url_ffhq), w=1024, h=1024, seeds=[91,388], psis=[1, 0.7, 0.5, 0, -0.5, -1])
    #draw_uncurated_result_figure(os.path.join(config.result_dir, 'figure10-uncurated-bedrooms.png'), load_Gs(url_bedrooms), cx=0, cy=0, cw=256, ch=256, rows=5, lods=[0,0,1,1,2,2,2], seed=0)
    #draw_uncurated_result_figure(os.path.join(config.result_dir, 'figure11-uncurated-cars.png'), load_Gs(url_cars), cx=0, cy=64, cw=512, ch=384, rows=4, lods=[0,1,2,2,3,3], seed=2)
    #draw_uncurated_result_figure(os.path.join(config.result_dir, 'figure12-uncurated-cats.png'), load_Gs(url_cats), cx=0, cy=0, cw=256, ch=256, rows=5, lods=[0,0,1,1,2,2,2], seed=1)

#----------------------------------------------------------------------------

if __name__ == "__main__":
    main()

#----------------------------------------------------------------------------


Instructions for updating:
Colocations handled automatically by placer.
/content/drive/My Drive/images/figure03-style-mixing.png
(4, 512)
(4, 18, 512)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [0]:
!ls /content/drive/My\ Drive/

'Colab Notebooks'				  example.png
'Copy of karras2019stylegan-ffhq-1024x1024.pkl'   projects
